In [ ]:
"""
대조군) 기본 학습값
실험군1) 기존 상하좌우 반전 (p=1이면 안됨) + 명도/대비 조정
실험군2) 데이터 증강 안 하고 그냥 9배로 복사만 한 경우
실험군3) 노이즈까지 추가한 경우 (coat_ar_noise.ipyb참조)
"""

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
run_control_base.py — 대조군 (증강 없음, 기본 학습값)
출력 : result_control_base.json  /  pred_control_base.csv
"""

# ───────── 0. import
import os, json, random, warnings
import numpy as np, pandas as pd
from PIL import Image, ImageFile
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings("ignore", category=UserWarning)

# ───────── 1. 하이퍼파라미터 / 경로 
SEED = 42
CSV_PATH = r"C:\Users\ast\Documents\project\train.csv"
IMG_DIR  = r"C:\Users\ast\Documents\project\train_images"
BATCH = 8
EPOCHS = 5
LR = 1e-4
WD = 0.01                     
SMOOTH = 0.0

OUT_MET = "result_control_base.json"
OUT_PRED = "pred_control_base.csv"
SPLIT_JSON = "split42.json"

# ───────── 2. 시드 & 멀티프로세싱
def seed_everything(s):
    random.seed(s); np.random.seed(s); os.environ["PYTHONHASHSEED"]=str(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False
seed_everything(SEED)
import torch.multiprocessing as mp
if mp.get_start_method(allow_none=True) != "spawn":
    mp.set_start_method("spawn", force=True)

# ───────── 3. Dataset
class ScrapDataset(Dataset):
    def __init__(self, df, img_dir, tf, label_enc):
        self.df  = df.reset_index(drop=True).copy()
        self.dir = img_dir
        self.tf  = tf
        self.df["cls"] = label_enc.transform(self.df["weight_class"])
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.dir, row.filename)).convert("RGB")
        img = self.tf(img)
        return img, torch.tensor(row.cls), row.filename

plain_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ───────── 4. split 고정
df = pd.read_csv(CSV_PATH)
if os.path.exists(SPLIT_JSON):
    idx = json.load(open(SPLIT_JSON)); train_idx, test_idx = idx["train"], idx["test"]
else:
    train_idx, test_idx = train_test_split(
        range(len(df)), test_size=0.2,
        stratify=df["weight_class"], random_state=SEED)
    json.dump({"train":train_idx, "test":test_idx}, open(SPLIT_JSON,"w"))
train_df, test_df = df.iloc[train_idx], df.iloc[test_idx]
le = LabelEncoder().fit(train_df["weight_class"])

train_loader = DataLoader(
    ScrapDataset(train_df, IMG_DIR, plain_tf, le),
    batch_size=BATCH, shuffle=True, num_workers=0)
test_loader  = DataLoader(
    ScrapDataset(test_df,  IMG_DIR, plain_tf, le),
    batch_size=BATCH, shuffle=False, num_workers=0)

# ───────── 5. 모델
class CoaTMedium(nn.Module):
    def __init__(self, n_cls):
        super().__init__()
        self.net = timm.create_model('coat_lite_medium',
                                     pretrained=True, num_classes=n_cls)
    def forward(self, x): return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = CoaTMedium(len(le.classes_)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
criterion  = nn.CrossEntropyLoss(label_smoothing=SMOOTH)

# ───────── 6. 학습
for ep in range(1, EPOCHS+1):
    model.train(); total=0
    for xb, yb, _ in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad(); loss = criterion(model(xb), yb)
        loss.backward(); optimizer.step()
        total += loss.item()*xb.size(0)
    print(f"[{ep}/{EPOCHS}] Loss={total/len(train_loader.dataset):.4f}")

# ───────── 7. 평가
model.eval(); yt, yp, rows = [], [], []
with torch.no_grad():
    for xb, yb, f in test_loader:
        pred = model(xb.to(device)).argmax(1).cpu()
        yt += yb.tolist(); yp += pred.tolist()
        rows += list(zip(f, le.inverse_transform(pred.numpy())))
acc = accuracy_score(yt, yp)
f1  = f1_score(yt, yp, average="macro")
print(f"✅ Control-Base  Acc={acc:.4f}  Macro-F1={f1:.4f}")

# ───────── 8. 저장
json.dump({"experiment":"control_base","accuracy":acc,"macro_f1":f1},
          open(OUT_MET,"w"), indent=2)
pd.DataFrame(rows, columns=["filename","predicted_label"]).to_csv(OUT_PRED, index=False)
print(f"📄 Metrics → {OUT_MET}\n📄 Preds   → {OUT_PRED}")


[1/5] Loss=1.1468
[2/5] Loss=0.6229
[3/5] Loss=0.2321
[4/5] Loss=0.0502
[5/5] Loss=0.0159
✅ Control-Base  Acc=0.4286  Macro-F1=0.4339
📄 Metrics → result_control_base.json
📄 Preds   → pred_control_base.csv


In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
run_control_base.py — 대조군 (증강 없음, 기본 학습값)
출력 : result_control_base.json  /  pred_control_base.csv
"""

# ─── 0. 기본 import
import os, json, random, math, warnings, numpy as np, pandas as pd
from PIL import Image, ImageFile
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings("ignore", category=UserWarning)

# ─── 1. 하이퍼파라미터 / 경로
SEED = 42
CSV  = r"C:\Users\ast\Documents\project\train.csv"
IMG  = r"C:\Users\ast\Documents\project\train_images"
BATCH  = 16         
EPOCHS = 10         
LR     = 1e-4
WD     = 3e-4       
SMOOTH = 0.05        

SPLIT  = "split42.json"
OUT_MET = "result_control_ep10_bs16.json"
OUT_PRE = "pred_control_ep10_bs16.csv"

# ─── 2. 시드 & 멀티프로세싱
def seed_all(s):
    random.seed(s); np.random.seed(s); os.environ["PYTHONHASHSEED"]=str(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False
seed_all(SEED)
import torch.multiprocessing as mp
if mp.get_start_method(allow_none=True) != "spawn":
    mp.set_start_method("spawn", force=True)

# ─── 3. Dataset
class ScrapDS(Dataset):
    def __init__(self, df, tf, le):
        self.df=df.reset_index(drop=True).copy()
        self.dir=IMG; self.tf=tf
        self.df["cls"]=le.transform(self.df["weight_class"])
    def __len__(self): return len(self.df)
    def __getitem__(self,i):
        row=self.df.iloc[i]
        img=Image.open(os.path.join(self.dir,row.filename)).convert("RGB")
        return self.tf(img), torch.tensor(row.cls), row.filename

plain_tf=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

# ─── 4. split 고정
df=pd.read_csv(CSV)
if os.path.exists(SPLIT):
    idx=json.load(open(SPLIT)); train_idx, test_idx=idx["train"], idx["test"]
else:
    train_idx, test_idx=train_test_split(range(len(df)), test_size=0.2,
        stratify=df["weight_class"], random_state=SEED)
    json.dump({"train":train_idx,"test":test_idx}, open(SPLIT,"w"))

tr_df, te_df = df.iloc[train_idx], df.iloc[test_idx]
le = LabelEncoder().fit(tr_df["weight_class"])

train_ld = DataLoader(ScrapDS(tr_df, plain_tf, le), batch_size=BATCH,
                      shuffle=True , num_workers=0)
test_ld  = DataLoader(ScrapDS(te_df, plain_tf, le), batch_size=BATCH,
                      shuffle=False, num_workers=0)

# ─── 5. 모델 & Optim & Scheduler
class CoaTMedium(nn.Module):
    def __init__(self, n_cls):
        super().__init__()
        # timm backbone
        self.net = timm.create_model(
            'coat_lite_medium',
            pretrained=True,
            num_classes=n_cls
        )

    def forward(self, x):
        return self.net(x)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=CoaTMedium(len(le.classes_)).to(device)
optimizer=torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

total_steps=len(train_ld)*EPOCHS
warmup_steps=len(train_ld)       # 1 epoch warm-up
def lr_lambda(step):
    if step < warmup_steps:
        return (step+1)/warmup_steps
    prog=(step-warmup_steps)/(total_steps-warmup_steps)
    return 0.5*(1+math.cos(math.pi*prog))
scheduler=torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

criterion = nn.CrossEntropyLoss(label_smoothing=SMOOTH)

# ─── 6. 학습
for ep in range(1,EPOCHS+1):
    model.train(); loss_sum=0
    for xb,yb,_ in train_ld:
        xb,yb=xb.to(device), yb.to(device)
        optimizer.zero_grad(); loss=criterion(model(xb), yb)
        loss.backward(); optimizer.step(); scheduler.step()
        loss_sum += loss.item()*xb.size(0)
    print(f"[{ep}/{EPOCHS}] Loss={loss_sum/len(train_ld.dataset):.4f}")

# ─── 7. 평가
model.eval(); yt,yp,rows=[],[],[]
with torch.no_grad():
    for xb,yb,f in test_ld:
        p=model(xb.to(device)).argmax(1).cpu()
        yt+=yb.tolist(); yp+=p.tolist()
        rows+=list(zip(f, le.inverse_transform(p.numpy())))
acc=accuracy_score(yt,yp)
f1 = f1_score(yt,yp,average="macro")
print(f"✅ Control-EP10-BS16  Acc={acc:.4f}  Macro-F1={f1:.4f}")

# ─── 8. 저장
json.dump({"experiment":"control_ep10_bs16","accuracy":acc,"macro_f1":f1},
          open(OUT_MET,"w"), indent=2)
pd.DataFrame(rows, columns=["filename","predicted_label"])\
  .to_csv(OUT_PRE, index=False)
print(f"📄 Metrics → {OUT_MET}\n📄 Preds   → {OUT_PRE}")


[1/10] Loss=1.1140
[2/10] Loss=0.6211
[3/10] Loss=0.3182
[4/10] Loss=0.2022
[5/10] Loss=0.1760
[6/10] Loss=0.1735
[7/10] Loss=0.1729
[8/10] Loss=0.1723
[9/10] Loss=0.1719
[10/10] Loss=0.1717
✅ Control-EP10-BS16  Acc=0.5238  Macro-F1=0.5235
📄 Metrics → result_control_ep10_bs16.json
📄 Preds   → pred_control_ep10_bs16.csv


In [12]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
run_exp1_flip_color_9x.py — 실험군 1
  • Train : 70개 원본 × 9배 = 630 이미지
            (Random H/V Flip 0.5 + ColorJitter 0.2/0.2)
  • Test  : 남은 30개 원본 그대로
  • split42_70-30.json 재사용, seed 42 고정
출력 : result_exp1_flip_color_9x.json / pred_exp1_flip_color_9x.csv
"""

# ───────── 0. import & 공통 설정
import os, json, random, math, warnings, numpy as np, pandas as pd
from PIL import Image, ImageFile
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings("ignore", category=UserWarning)

# ───────── 1. 하이퍼파라미터 (대조군과 동일)
SEED   = 42
CSV    = r"C:\Users\ast\Documents\project\train.csv"
IMG    = r"C:\Users\ast\Documents\project\train_images"
BATCH  = 16
EPOCHS = 10
LR     = 1e-4
WD     = 3e-4
SMOOTH = 0.05
SPLIT  = "split42_70-30.json"           
OUT_MET= "result_exp1_flip_color_9x.json"
OUT_PRE= "pred_exp1_flip_color_9x.csv"

# 시드 고정
def seed_all(s):
    random.seed(s); np.random.seed(s); os.environ["PYTHONHASHSEED"]=str(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False
seed_all(SEED)
import torch.multiprocessing as mp
if mp.get_start_method(allow_none=True) != "spawn":
    mp.set_start_method("spawn", force=True)

# ───────── 2. Dataset 정의
class ScrapDS(Dataset):
    def __init__(self, df, tf, le):
        self.df=df.reset_index(drop=True).copy()
        self.dir=IMG; self.tf=tf
        self.df["cls"]=le.transform(self.df["weight_class"])
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row=self.df.iloc[i]
        img=Image.open(os.path.join(self.dir, row.filename)).convert("RGB")
        return self.tf(img), torch.tensor(row.cls), row.filename

# ───────── 3. Transform
aug_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
plain_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ───────── 4. split 70 / 30 고정
df = pd.read_csv(CSV)
if os.path.exists(SPLIT):
    idx=json.load(open(SPLIT)); train_idx, test_idx=idx["train"], idx["test"]
else:
    train_idx, test_idx = train_test_split(
        range(len(df)), train_size=70, test_size=30,
        stratify=df["weight_class"], random_state=SEED)
    json.dump({"train":train_idx, "test":test_idx}, open(SPLIT,"w"))

tr_df, te_df = df.iloc[train_idx], df.iloc[test_idx]
le = LabelEncoder().fit(tr_df["weight_class"])

# ───────── 5. Train / Test DataLoader
base_train_ds = ScrapDS(tr_df, aug_tf, le)
train_ds      = ConcatDataset([base_train_ds]*9)   # 70 × 9 = 630
test_ds       = ScrapDS(te_df, plain_tf, le)

train_ld = DataLoader(train_ds, batch_size=BATCH, shuffle=True , num_workers=0)
test_ld  = DataLoader(test_ds , batch_size=BATCH, shuffle=False, num_workers=0)

# ───────── 6. 모델, Optim, Scheduler
class CoaTMedium(nn.Module):
    def __init__(self, n_cls):
        super().__init__()
        # timm backbone ─ 들여쓰기 8칸(=4스페이스×2)
        self.net = timm.create_model(
            'coat_lite_medium',
            pretrained=True,
            num_classes=n_cls
        )

    def forward(self, x):
        return self.net(x)


device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CoaTMedium(len(le.classes_)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

total_steps=len(train_ld)*EPOCHS
warmup_steps=len(train_ld)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lambda step: (step+1)/warmup_steps if step<warmup_steps
                 else 0.5*(1+math.cos(math.pi*(step-warmup_steps)/(total_steps-warmup_steps)))
)
criterion = nn.CrossEntropyLoss(label_smoothing=SMOOTH)

# ───────── 7. 학습
for ep in range(1,EPOCHS+1):
    model.train(); loss_sum=0
    for xb,yb,_ in train_ld:
        xb,yb=xb.to(device), yb.to(device)
        optimizer.zero_grad(); loss=criterion(model(xb), yb)
        loss.backward(); optimizer.step(); scheduler.step()
        loss_sum += loss.item()*xb.size(0)
    print(f"[{ep}/{EPOCHS}] Loss={loss_sum/len(train_ld.dataset):.4f}")

# ───────── 8. 평가
model.eval(); yt,yp,rows=[],[],[]
with torch.no_grad():
    for xb,yb,f in test_ld:
        p=model(xb.to(device)).argmax(1).cpu()
        yt+= yb.tolist(); yp+= p.tolist()
        rows+=list(zip(f, le.inverse_transform(p.numpy())))
acc = accuracy_score(yt,yp)
f1  = f1_score(yt,yp,average="macro")
print(f"✅ Exp1-FlipColor-9x  Acc={acc:.4f}  Macro-F1={f1:.4f}")

# ───────── 9. 저장
json.dump({"experiment":"exp1_flip_color_9x","accuracy":acc,"macro_f1":f1},
          open(OUT_MET,"w"), indent=2)
pd.DataFrame(rows, columns=["filename","predicted_label"])\
  .to_csv(OUT_PRE, index=False)
print(f"📄 Metrics → {OUT_MET}\n📄 Preds   → {OUT_PRE}")


[1/10] Loss=0.5999
[2/10] Loss=0.1821
[3/10] Loss=0.1726
[4/10] Loss=0.1703
[5/10] Loss=0.1698
[6/10] Loss=0.1696
[7/10] Loss=0.1695
[8/10] Loss=0.1695
[9/10] Loss=0.1695
[10/10] Loss=0.1694
✅ Exp1-FlipColor-9x  Acc=0.4667  Macro-F1=0.4685
📄 Metrics → result_exp1_flip_color_9x.json
📄 Preds   → pred_exp1_flip_color_9x.csv


In [14]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
run_exp2_copy_9x.py — 실험군 2
  · Train : 70개 원본 × 9배 복제 = 630장 (증강 없음)
  · Test  : split42_70-30.json 에 남은 30장
  · Hyper : BATCH 16 · EPOCHS 10 · lr 1e-4 · wd 3e-4 · smoothing 0.05
출력 : result_exp2_copy_9x.json / pred_exp2_copy_9x.csv
"""

# ── 0. 기본 import ────────────────────────────────────────────────
import os, json, random, math, warnings, numpy as np, pandas as pd
from PIL import Image, ImageFile
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings("ignore", category=UserWarning)

# ── 1. 설정값 ────────────────────────────────────────────────────
SEED   = 42
CSV    = r"C:\Users\ast\Documents\project\train.csv"
IMG    = r"C:\Users\ast\Documents\project\train_images"
BATCH  = 16
EPOCHS = 10
LR     = 1e-4
WD     = 3e-4
SMOOTH = 0.05

SPLIT  = "split42_70-30.json"
OUT_MET= "result_exp2_copy_9x.json"
OUT_PRE= "pred_exp2_copy_9x.csv"

# ── 2. 시드 & 멀티프로세싱 고정 ───────────────────────────────────
def seed_all(s):
    random.seed(s); np.random.seed(s); os.environ["PYTHONHASHSEED"]=str(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False
seed_all(SEED)
import torch.multiprocessing as mp
if mp.get_start_method(allow_none=True) != "spawn":
    mp.set_start_method("spawn", force=True)

# ── 3. Dataset 정의 ──────────────────────────────────────────────
class ScrapDataset(Dataset):
    def __init__(self, df, tf, le):
        self.df  = df.reset_index(drop=True).copy()
        self.dir = IMG
        self.tf  = tf
        self.df["cls"] = le.transform(self.df["weight_class"])
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.dir, row.filename)).convert("RGB")
        return self.tf(img), torch.tensor(row.cls), row.filename

plain_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ── 4. 70/30 split 고정 ──────────────────────────────────────────
df = pd.read_csv(CSV)
if os.path.exists(SPLIT):
    idx = json.load(open(SPLIT)); train_idx, test_idx = idx["train"], idx["test"]
else:
    train_idx, test_idx = train_test_split(
        range(len(df)), train_size=70, test_size=30,
        stratify=df["weight_class"], random_state=SEED)
    json.dump({"train":train_idx, "test":test_idx}, open(SPLIT,"w"))

tr_df, te_df = df.iloc[train_idx], df.iloc[test_idx]
le = LabelEncoder().fit(tr_df["weight_class"])

# ── 5. DataLoader (원본 9× 복제) ─────────────────────────────────
base_train_ds = ScrapDataset(tr_df, plain_tf, le)       # 증강 X
train_ds      = ConcatDataset([base_train_ds]*9)        # 70 × 9 = 630
test_ds       = ScrapDataset(te_df, plain_tf, le)

train_ld = DataLoader(train_ds, batch_size=BATCH, shuffle=True , num_workers=0)
test_ld  = DataLoader(test_ds , batch_size=BATCH, shuffle=False, num_workers=0)

# ── 6. 모델 & Optim & 스케줄러 ──────────────────────────────────
class CoaTMedium(nn.Module):
    def __init__(self, n_cls):
        super().__init__()
        self.net = timm.create_model('coat_lite_medium',
                                     pretrained=True, num_classes=n_cls)
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = CoaTMedium(len(le.classes_)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

total_steps = len(train_ld) * EPOCHS
warmup_steps = len(train_ld)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lr_lambda=lambda step: (step+1)/warmup_steps if step < warmup_steps
             else 0.5*(1+math.cos(math.pi*(step-warmup_steps)/(total_steps-warmup_steps)))
)
criterion = nn.CrossEntropyLoss(label_smoothing=SMOOTH)

# ── 7. 학습 루프 ────────────────────────────────────────────────
for ep in range(1, EPOCHS+1):
    model.train(); epoch_loss = 0.0
    for xb, yb, _ in train_ld:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward(); optimizer.step(); scheduler.step()
        epoch_loss += loss.item() * xb.size(0)
    print(f"[{ep}/{EPOCHS}] Loss={epoch_loss/len(train_ld.dataset):.4f}")

# ── 8. 평가 ────────────────────────────────────────────────────
model.eval(); yt, yp, rows = [], [], []
with torch.no_grad():
    for xb, yb, f in test_ld:
        preds = model(xb.to(device)).argmax(1).cpu()
        yt += yb.tolist(); yp += preds.tolist()
        rows += list(zip(f, le.inverse_transform(preds.numpy())))
acc = accuracy_score(yt, yp)
f1  = f1_score(yt, yp, average="macro")
print(f"✅ Exp2-Copy9x  Acc={acc:.4f}  Macro-F1={f1:.4f}")

# ── 9. 결과 저장 ───────────────────────────────────────────────
json.dump({"experiment":"exp2_copy_9x","accuracy":acc,"macro_f1":f1},
          open(OUT_MET,"w"), indent=2)
pd.DataFrame(rows, columns=["filename","predicted_label"])\
  .to_csv(OUT_PRE, index=False)
print(f"📄 Metrics → {OUT_MET}\n📄 Preds   → {OUT_PRE}")


[1/10] Loss=0.5019
[2/10] Loss=0.1720
[3/10] Loss=0.1695
[4/10] Loss=0.1693
[5/10] Loss=0.1693
[6/10] Loss=0.1693
[7/10] Loss=0.1692
[8/10] Loss=0.1692
[9/10] Loss=0.1692
[10/10] Loss=0.1692
✅ Exp2-Copy9x  Acc=0.5000  Macro-F1=0.5018
📄 Metrics → result_exp2_copy_9x.json
📄 Preds   → pred_exp2_copy_9x.csv
